# Libraries


In [47]:
import requests 
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import os
import re
from bs4 import BeautifulSoup 

# Static Variables


In [48]:
base_url = "https://www.holy-bhagavad-gita.org"

# Fetch All Chapters as html


In [49]:
all_chapters_as_html = requests.get(f"{base_url}/index") 
soup = BeautifulSoup(all_chapters_as_html.content, 'html5lib')

In [50]:
chapter_sctions = soup.findAll('section')
list_of_chapters = []
for chapter_section in chapter_sctions:
    list_of_chapters.append(
        {
            "Chapter": chapter_section.a.text,
            "url": f"{base_url}{chapter_section.a['href']}"
        }
    )

In [51]:
for i in range(len(list_of_chapters)):
    all_chapters_as_html = requests.get(list_of_chapters[i]["url"]) 
    current_chapter = BeautifulSoup(all_chapters_as_html.content, 'html5lib')
    this_dict = list_of_chapters[i]
    this_dict["chapter_soup"] = current_chapter
    list_of_chapters[i] = this_dict

# Fetch All verses of a Chapter


In [ ]:
def filter_strings(mystr: str):
    # all_puntuation = """"#$'()*+./;<=>@[\]^_`}{~"""
    # my_line = re.sub(
    #     rf"[{all_puntuation}]",
    #     "", 
    #     mystr
    # )

    # my_line = my_line.replace("\r", "")
    # my_line = my_line.replace("\t", "")
    # my_line = my_line.replace("\n", "")
    # my_line = my_line.strip()
    return mystr
def fetch_verse_content(verse_link: str):

    def fetch_info(my_elm, attr):
        try:
            my_line = str(my_chapter_verse_soup.find(my_elm, attrs=attr).text).strip()
            
            return filter_strings(my_line)
        except Exception:
            return "Not Available !!!"
    chapter_verse = requests.get(verse_link)
    my_chapter_verse_soup = BeautifulSoup(chapter_verse.content, 'html5lib')
    original_verse = fetch_info("div", {"id":"originalVerse"})
    transliteration_verse = fetch_info("div", {"id":"transliteration"})
    wordMeanings_verse = fetch_info("div", {"id":"wordMeanings"})
    transliteration_wo_dia_verse = fetch_info("div", {"id":"transliteration_wo_dia"})
    verse_translation = fetch_info("div",{"id":"translation"})
    verse_commentary = fetch_info("div", {"id":"commentary"})

    return {
        "original_verse": original_verse,
        "transliteration_verse": transliteration_verse,
        "wordMeanings_verse": wordMeanings_verse,
        "transliteration_wo_dia_verse": transliteration_wo_dia_verse,
        "verse_translation": verse_translation,
        "verse_commentary": verse_commentary
    }

dataframe_set = []

def fetch_chapter_content(chapter_url: str, current_language: str, chapter_name):
    chapter_content = requests.get(chapter_url)
    my_chapter_soup = BeautifulSoup(chapter_content.content, 'html5lib')
    my_chapter_intro = str(my_chapter_soup.find('div', attrs={"class":"chapterIntro"}).text).strip()
    my_chapter_verses = my_chapter_soup.find('div', attrs={"class":"listItems"})
    my_chapter_verses = my_chapter_verses.find_all('a')
    my_verse_info = []
    for verse in my_chapter_verses:
        my_text = str(verse.find_all('span')[0].text).strip()
        my_dict = fetch_verse_content(f'{base_url}{verse["href"]}')
        my_dict["my_chapter_intro"] = filter_strings(my_chapter_intro)
        my_dict["current_language"] = current_language
        my_verse_info.append(my_dict)
        print("Processing verse: ", verse["href"], "...")

    my_df = pd.DataFrame(my_verse_info)

    dataframe_set.append({
        "chapter": chapter_name,
        "verse_dataframe": my_df
    })

for current_chapter in list_of_chapters:
    my_soup = current_chapter["chapter_soup"]
    my_languages_soup = my_soup.findAll('div', attrs = {'align':'right'})[0]
    my_languages_soup = my_languages_soup.findAll('li')
    
    for lis in my_languages_soup:
        all_as = lis.find('a')
        chapter_url_language = all_as["href"]
        current_language = str(lis["id"]).strip()
        chapter_name = str(current_chapter["Chapter"]).strip().replace(" ","_")
        
        print("Processing Chapter: ", chapter_name, " Language: ", current_language)
        fetch_chapter_content(
            chapter_url_language, 
            current_language,
            chapter_name
        )

In [66]:
base_location = "D:\development\output\\bhagwad_gita"

for elem in dataframe_set:
    my_path = os.path.join(base_location, elem["chapter"])
    if(os.path.exists(my_path) == False):
        os.mkdir(my_path)
    my_current_df = elem["verse_dataframe"]
    language = str(my_current_df["current_language"][0])
    my_unique_file_name = f"{elem['chapter']}_{language}.xlsx"
    filename = os.path.join(my_path, my_unique_file_name)
    with pd.ExcelWriter(filename) as writer:
        my_current_df.to_excel(writer, index=False, engine='xlsxwriter')